## To Do

In [16]:
# To Do
# Organize the Cleaning code into readable groups
# Document the cleaning process
# Checking datatypes of both sets
# Considering renaming any columns (LOCATION, TIME)
# Checking for data loss thorughout the cleaning process

## Requirements

In [17]:
# Importing Dependencies
import pandas as pd
import os
import csv
import chardet

In [18]:
# The WHO Mortaity Dataset

# Combine Mortality rate files into a single dataframe
dirLoc = 'Resources' #filepath of the directory
Mortality_df = pd.DataFrame()

for file in os.scandir(dirLoc):
    if "Morti" in file.path: # Only append files from the Mortality Datasets
        Mortality_df = Mortality_df.append(pd.read_csv(file.path,)) #append the data of the current mortality file being read

c:\Users\daypa\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (4) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,
c:\Users\daypa\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (2) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [19]:
# Reading in the OECD Meat Consumption dataset
OECD_df = pd.read_csv('Resources/OECD_Meat_Consumption.csv')

## Data Exploration

Summary:
* Scope of the datasets (time)
* Irrelevant fields
* Irrelevant records and null values
* Data types
* Column Renaming
* Full Country Name Mapping

### Time range of dataset

In [20]:
# Investigating the time the datasets encompass

# The Mortaility rate dataset
Mortality_max = Mortality_df['Year'].max()
Mortality_min = Mortality_df['Year'].min()
print(f'The earliest year in the mortality dataset is {Mortality_min} and the latest year is {Mortality_max}')

OECD_max = OECD_df['TIME'].max()
OECD_min = OECD_df['TIME'].min()
print(f'The earliest year in the OECD dataset is {OECD_min} and the latest year is {OECD_max}')

# We want to narrow the scope of our records between the year 2000 and 2020
Mortality_df = Mortality_df[Mortality_df['Year'] >= 2000]

OECD_df = OECD_df[OECD_df['TIME'] >= 2000]
OECD_df = OECD_df[OECD_df['TIME'] <= 2020]

The earliest year in the mortality dataset is 1988 and the latest year is 2020
The earliest year in the OECD dataset is 1990 and the latest year is 2029


### Irrelevant Fields

In [21]:
# Limiting the field in the mortality dataset to: ["Country","Year","Cause","Sex","Deaths1"]
Mortality_df = Mortality_df[["Country","Year","Sex","Deaths1"]]

# Limiting the OECD dataset to the fields: ['LOCATION', 'SUBJECT', 'MEASURE','TIME','Value']
OECD_df = OECD_df[['LOCATION', 'SUBJECT', 'MEASURE','TIME','Value']]

Irrelevant Records

In [22]:
# Mortality Dataset
# Removing records with value 9
Mortality_df["Sex"].value_counts() # There are 20,701 records with the 9 value in the sex field which refers to other

# Creating new dataframe wihtout 9 values in the Sex field
Mortality_df = Mortality_df[Mortality_df["Sex"] != 9]

Mortality_df["Sex"].value_counts() # 9 values removed from the original dataset

code_to_gender = {1: 'M' , 2: 'F'}

Mortality_df['Sex'] = Mortality_df['Sex'].map(code_to_gender)



print(Mortality_df.count()) # There are no null values

Country    4039092
Year       4039092
Sex        4039092
Deaths1    4039092
dtype: int64


In [23]:
# OECD Dataset

OECD_df['MEASURE'].unique() # Dataset contains records in two units

OECD_df = OECD_df[OECD_df['MEASURE'] != 'THND_TONNE'] # Filtering out values measured in Thousand tonnes

OECD_df.count() # No null values present

LOCATION    3192
SUBJECT     3192
MEASURE     3192
TIME        3192
Value       3192
dtype: int64

In [24]:
#Mortality_df['Cause'] = 'Deaths'

### Data types

In [25]:
Mortality_df.dtypes # Columns Cause, Sex, and Country dont seem to be the datatype we need them to be

# We can explicitly assign the string datatype to the Cause column
Mortality_type_Dict = {'Country': 'string' , 'Sex':'string'}
Mortality_df = Mortality_df.astype(Mortality_type_Dict)


Mortality_df.dtypes # The Cause Column is now string

Country    string
Year        int64
Sex        string
Deaths1     int64
dtype: object

In [26]:
OECD_df.dtypes # LOCATION, SUBJECT, AND MEASURE should be string
OECD_type_dict = {'LOCATION':'string', 'SUBJECT':'string', 'MEASURE':'string'}
OECD_df = OECD_df.astype(OECD_type_dict)
OECD_df.dtypes

LOCATION     string
SUBJECT      string
MEASURE      string
TIME          int64
Value       float64
dtype: object

### Renaming Columns in OECD

In [27]:
OECD_df.rename(columns = {'LOCATION': 'Country','TIME':'Year'}, inplace=True)

### Country Name Mapping

In [28]:
# mapping the country name to Mortality_df using the WHO_Country_Map.csv file
# Reading in the WHO country map as a dictionary

WHO_Code_To_Country = {}

with open('Resources/WHO_Country_Map.csv', mode = 'r') as file:
    reader = csv.reader(file)
    WHO_Code_To_Country = {rows[0] : rows[1] for rows in reader}


WHO_Code_To_Country.pop('country') # removing the header from the dictionary

#apply the WHO_Code_To_Country map to the "Country" series in Mortality_df
Mortality_df['Country'] = Mortality_df['Country'].map(WHO_Code_To_Country)

In [29]:
# mapping 3 character country code to full country name
ISOMap_File = 'Resources/ISO_Alpha3_Country_Codes.csv'
Alpha3_To_Country = {}

# Unknown encoiding, use chardetect to find correct encoding to use
with open(ISOMap_File, 'rb') as chardetect:
    print(chardet.detect(chardetect.read()))  #Determine the encoding used in the file with chardet


with open('Resources/ISO_Alpha3_Country_Codes.csv', 'r', encoding = 'cp1252') as file:
    reader = csv.reader(file)
    Alpha3_To_Country = {row[1] : row[0] for row in reader } # read in the file using cp1252 encoding

Alpha3_To_Country.pop('Alpha-3 code') # remove the header from the dictionary

#Swap out the three letter codes with the country name
OECD_df['Country'] = OECD_df['Country'].map(Alpha3_To_Country)



{'encoding': 'Windows-1252', 'confidence': 0.73, 'language': ''}


### Dimensionality reduction


In [33]:
# Consolidating causes of death as one
reduced_mortality = Mortality_df.groupby(['Country' , 'Year']).sum()

reduced_mortality = reduced_mortality.reset_index(level = ['Country', 'Year'] )
reduced_mortality.head()

,Country,Year,Deaths1
0,Andorra,2011,794
1,Andorra,2012,880
2,Andorra,2013,698
3,Andorra,2014,797
4,Andorra,2015,830


In [34]:
# Consolidating value by SUBJECT
reduced_OECD = OECD_df.groupby(['Country' , 'Year','MEASURE']).sum()

reduced_OECD = reduced_OECD.reset_index(level = ['Country','Year','MEASURE'] )

reduced_OECD.head()


,Country,Year,MEASURE,Value
0,Argentina,2000,KG_CAP,75.585
1,Argentina,2001,KG_CAP,72.192
2,Argentina,2002,KG_CAP,61.737
3,Argentina,2003,KG_CAP,63.245
4,Argentina,2004,KG_CAP,68.114


### Merging the datasets

In [35]:
# merging the OECD and WHO Dataset
reduced_OECD #3192 records
reduced_mortality #4039092 records

lethalMeat_df = reduced_mortality.merge(right=reduced_OECD , how='inner' , on= ['Country','Year'])

In [36]:
# replace NaN with values we can work with 'Missing'
len(lethalMeat_df)

386

In [37]:
# Writing the dataset as a CSV
lethalMeat_df.to_csv('Resources/lethal_meat.csv')

In [38]:
lethalMeat_df.head()

,Country,Year,Deaths1,MEASURE,Value
0,Argentina,2000,554178,KG_CAP,75.585
1,Argentina,2001,571412,KG_CAP,72.192
2,Argentina,2002,581740,KG_CAP,61.737
3,Argentina,2003,603266,KG_CAP,63.245
4,Argentina,2004,587300,KG_CAP,68.114
